# Detección de movimiento por diferencias

- Idea base: si dos imágenes consecutivas son casi iguales, su diferencia será cercana a cero.

- Movimiento = cambio en intensidad de píxeles.

- El umbral separa ruido o pequeñas variaciones de cambios reales.

- El blur reduce falsos positivos debidos al ruido del sensor o compresión.

- Dilatación une regiones dispersas (refuerza la “máscara” de movimiento).

In [1]:
import cv2

# Captura de video (0 = cámara)
cap = cv2.VideoCapture(0)

# Leer el primer frame
ret, frame1 = cap.read()
if not ret:
    print("Error: no se pudo acceder a la cámara.")
    cap.release()
    exit()

# Convertir a gris y suavizar
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray1 = cv2.GaussianBlur(gray1, (21, 21), 0)

while True:
    ret, frame2 = cap.read()
    if not ret:
        break

    # Procesar frame actual
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.GaussianBlur(gray2, (21, 21), 0)

    # Diferencia absoluta entre frames
    diff = cv2.absdiff(gray1, gray2)

    # Umbralización
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Dilatar para eliminar pequeños huecos
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Buscar contornos del movimiento detectado
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    movimiento_detectado = False

    for contour in contours:
        if cv2.contourArea(contour) < 1000:
            continue  # ignorar ruido pequeño
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame2, (x, y), (x+w, y+h), (0, 255, 0), 2)
        movimiento_detectado = True

    estado = "MOVIMIENTO" if movimiento_detectado else "QUIETO"
    color_estado = (0, 0, 255) if movimiento_detectado else (255, 255, 255)
    cv2.putText(frame2, estado, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color_estado, 3)

    # Mostrar resultados
    cv2.imshow("Frame", frame2)
    cv2.imshow("Diferencia", thresh)

    # Actualizar frame anterior
    gray1 = gray2.copy()

    if cv2.waitKey(30) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()
